In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import random
import os
import openpyxl

from Betano import *
from Portal import *
from Sky_Under import *

#se der erro pode ser por estes dois
from big_dix import *
from Multi_DeForest_Husky_C import *

import Add_HUSKY as H1
time.sleep(0.5)
#------------------------- E que comece o código
#Get the dfs

#Betano
downloads = 'C://Users//joaom//Downloads//'
desktop = 'C://Users//joaom//Desktop//WScrapy2//Sky//'

print('Which Leagues?')
print('(all -> All the Leagues || done -> No more Leagues)')
leagues_aux = 0
leagues_aux2 = 0 #serve para não ter de fazer raise quando me engano no input
leagues_add = []
while leagues_aux not in ['all','done']:
    if (leagues_aux!=0) and (leagues_aux2!=1):
        leagues_add+=[leagues_aux]
    print(['Premier_League','Bundesliga','Ligue_1','La_Liga','Serie_A','Eredivisie','Premiership'])
    time.sleep(0.2)
    leagues_aux = input_string_aux(input('--------->'))
    if (leagues_aux in leagues_add) and (leagues_aux not in ['all','done']) and (leagues_aux not in ['Premier_League','Bundesliga','Ligue_1','La_Liga','Serie_A','Eredivisie','Premiership']):
        print('Argument not valid')
        leagues_aux2 = 1

if leagues_aux=='all':
    leagues = ['Premier_League','Bundesliga','Ligue_1','La_Liga','Serie_A','Eredivisie','Premiership']
else:
    leagues = leagues_add

print(leagues,'L')

print('Make Predictions?')
is_predictions = int(input('0(No) / 1(Yes) :  '))
if is_predictions not in [0,1]:
    raise Exception('Argument not valid')
print('Here')

B = list(map(lambda x: 'betano' in x,list(os.listdir(downloads))))
if sum(B)!=1: #True==1 || False==0
    raise Exception('Erro na extração do Betano, Temos '+str(sum(B))+' ocorrências')
else:
    Bet_ind=np.where(B)[0][0]
    betano = pd.read_csv(downloads+os.listdir(downloads)[Bet_ind])

#Portal
P = list(map(lambda x: 'oddsportal' in x,list(os.listdir(downloads))))
if sum(P)!=1: #True==1 || False==0
    raise Exception('Erro na extração do Portal, Temos '+str(sum(P))+' ocorrências')
else:
    Portal_ind=np.where(P)[0][0]
    portal = pd.read_csv(downloads+os.listdir(downloads)[Portal_ind])


#Under
U = list(map(lambda x: 'understats' in x,list(os.listdir(downloads))))
if sum(U)!=1: #True==1 || False==0
    raise Exception('Erro na extração do Under, Temos '+str(sum(U))+' ocorrências')
else:
    Under_ind=np.where(U)[0][0]
    under = pd.read_csv(downloads+os.listdir(downloads)[Under_ind])


#Sky
S1 = list(map(lambda x: 'skysports' in x,list(os.listdir(downloads))))
if sum(S1)!=1:
    S2 = list(map(lambda x: 'skygames' in x,list(os.listdir(desktop))))
    if sum(S2)!=1:
        raise Exception('Erro na extração do Sky, Ocorrências: '+str(sum(S1))+' (Downloads) || '+str(sum(S2))+' (Desktop)')
    else:
        Sky_ind=np.where(S2)[0][0]
        sky = pd.read_csv(desktop+os.listdir(desktop)[Sky_ind])
else:
    Sky_ind=np.where(S1)[0][0]
    sky = pd.read_csv(downloads+os.listdir(downloads)[Sky_ind])

#Filtering the dataaaaaa
#Betano
Bdir = 'C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//' 
save_rawBetano(betano)
Betano_autofilter()
betano_dfs = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
betano1 = Merge_Betano(betano_dfs)

print('Here3')
#Portal
print('Portal')
Pdir = 'C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Portal_Info//Filtered_Data//' #dps se calhar também podemos fazer merge
save_rawPortal(portal)
Portal_autofilter()
portal0 = portal_drop(portal)


#Sky
print('Skyyyyyyyyyyyyyyyyyyyyyyyyyyyy')
Sdir = 'C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Sky_Info//Filtered_Data//'  #dps se calhar também podemos fazer merge
save_rawSky(sky)
Sky_autofilter()
sky_dfs = list(map(lambda x:pd.read_csv(Sdir+x),list(os.listdir(Sdir))))
sky1 = merge_SU(sky_dfs)


#Under
print('Undeeeeeeeeeeeer')
Udir = 'C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Under_Info//Filtered_Data//'  #dps se calhar também podemos fazer merge
save_rawUnder(under)
Under_autofilter()
under_dfs = list(map(lambda x:pd.read_csv(Udir+x),list(os.listdir(Udir))))
under1 = merge_SU(under_dfs)



#IT´s TIME !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#leagues=['Premier_League','Bundesliga','Ligue_1','La_Liga','Serie_A','Eredivisie','Premiership']
#Fazer depois o código para os Russos!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

not_add = pd.DataFrame({}) #vai ajudar a dizer qnd é q temos de fazer scrape
for l in leagues:
    season = int(input('Which season?:   '))
    if len(str(season))!=4:
        raise Exception('Season '+str(season)+' não é válida')
    
    if l in ['Premier_League','Bundesliga','Ligue_1','La_Liga','Serie_A']:
        sky2 = Prepare__Df(sky1,season,l)
        under2 = Prepare__Df(under1,season,l)
        portal2 = Prepare__Df(portal0,season,l)
        betano2 =Prepare__Df(betano1,season,l)

        husky,lixo0 = Merge_Sky_Under(sky2,under2)

        lixo1,portal3 = Portal_utn(sky2,portal2,l)
        lixo2,betano3 = Betano_utn(sky2,betano2,l)


        portal_add,portal_not_add,portal_len = PortalAdd_Sky2(portal3,husky)

        not_add = pd.concat([not_add,portal_not_add],sort=False)

        husky0 = pd.concat([husky,portal_add],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)


        PreHusky= Betano_addodds(husky0,betano3)
    elif l in ['Eredivisie','Premiership']:
        sky2 = Prepare__Df(sky1,season,l)
        
        portal2 = Prepare__Df(portal0,season,l)
        betano2 =Prepare__Df(betano1,season,l)            
        lixo1,portal3 = Portal_utn(sky2,portal2,l)
        print('Portal is OK----------------')
        lixo2,betano3 = Betano_utn(sky2,betano2,l)
        sky3 = Prepare__Df(sky1,season,l)
        portal_add,portal_not_add,portal_len = PortalAdd_Sky2(portal3,sk3)
        not_add = pd.concat([not_add,portal_not_add],sort=False)
        sky2 = pd.concat([sky2,portal_add],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
        PreHusky= Betano_addodds(sky2,betano3)

    if is_predictions==1:

        All_Predix = pd.DataFrame({})

        if l == 'Ligue_1': #o modelo e os valores limite estão errados
            modelx = 'Husky_DFLL1_P2.pickle'
            TL = 60
            FL = 60
            clubs = H1.findall_clubs2_new(PreHusky,read=False)
            Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
            Husky = Husky0.tail(portal_len)
            rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
            F=pickle.load(rd0)
            lixo_Train=pickle.load(rd0)
            lixo_Test=pickle.load(rd0)
            T_df=pickle.load(rd0)
            rd0.close()
            print(Husky.columns)
            big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
            Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
            Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
            betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
            Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
            lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
            Predix=Odds_Top(Predix,Betano_TopOdds)
        
        
        # os valores dos thresholds dependem de modelo para mkdelo e cuidado q eles estão no drop clumns
        All_Predix = pd.concat([All_Predix,Predix],sort=False)
    League_Pref = {
    'Premier_League':'PL','La_Liga':'LL','Ligue_1':'L','Serie_A':'SA','Bundesliga':'Bund','Eredivisie':'Ered',
        'Premiership':'Scott'
    }
    PreHusky.to_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//ADF//'+str(l)+'//New'+str(League_Pref[l])+str(str(season-1)[2:])+'_'+str(str(season)[2:])+'.xlsx',index=False)



# FIM
if is_predictions==1:
    All_Predix = All_Predix.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    All_Predix.to_excel('All_Predix.xlsx',index=False)
not_add.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

No final da época 19/20, como houve exceções temos de criar funções excecionais para obter os jogos de cada ano
big_dix tem funções para analisar os modelos
Este módulo usa pandas.Dataframe
E está feito para apenas prever variáveis categóricas 0/1
 
No DeForest, escolhemos as colunas a analisar por árvore
Ao contrário do Random_Forest, onde fazíamos random
Neste modelos vamos fazer uma experiência e não usar Odds
Em vex disso, usamos os xPTS para calcular o "power" de uma equipa
Which Leagues?
(all -> All the Leagues || done -> No more Leagues)
['Premier_League', 'Bundesliga', 'Ligue_1', 'La_Liga', 'Serie_A', 'Eredivisie', 'Premiership']
--------->all
['Premier_League', 'Bundesliga', 'Ligue_1', 'La_Liga', 'Serie_A', 'Eredivisie', 'Premiership'] L
Make Predictions?
0(No) / 1(Yes) :  0
Here
[76, 87, 149] NaN Indexes
Ligue 1 1
problema com 25
---------------
Ligue 1 1
problema com BS
---------------
Championship 5
problema com 25
---------------
Championship 5
problema com BS
------------

Which season?:   2021
Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"
Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"
Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"
Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"
What
sky 18 under 18
0 of  18
s 0
1 of  18
s 4
2 of  18
s 1
3 of  18
s 2
4 of  18
s 3
5 of  18
s 5
6 of  18
s 6
7 of  18
s 7
8 of  18
s 8
9 of  18
s 9
10 of  18
s 12
11 of  18
s 10
12 of  18
s 15
13 of  18
s 14
14 of  18
s 11
15 of  18
s 13
16 of  18
s 16
17 of  18
s 17
last here we go
original_size: sky 18 | under 18
no_find []
file_size 18
SKYyy ['Stuttgart', 'SC Freiburg', 'Werder Bremen', 'Hertha Berlin', 'RB Leipzig', 'Arminia Bielefeld', 'Bayern Munich', 'Borussia Dortmund', 'Hoffenheim', 'Eintracht Frankfurt', 'Schalke', 'Wolfsburg', 'FC Augsburg', "M'gladbach", 'Cologne', 'Mainz', '1. FC Union Berlin', 'Bayer Leverkusen']
Under ['

NameError: name 'sk3' is not defined

In [8]:
Bdir = 'C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//' 
br = [b for b in os.listdir(Bdir) if 'betano' in b and ('Pre' not in b) and ('csv' in b)]
print(len(br))
for bet in br:
    aa=pd.read_csv(Bdir+bet)
    for i in range(len(aa)):
        if aa.iloc[i]['HomeTeam']=='FC Zwolle':
            aa.at[i,'HomeTeam']='PEC Zwolle'
        elif aa.iloc[i]['AwayTeam']=='FC Zwolle':
            aa.at[i,'AwayTeam']='PEC Zwolle'
    aa.to_csv(Bdir+bet)

42


In [30]:
for i in range(len(b23)):
    if b23.iloc[i]['HT']=='FC Zwolle':
        print('ok')
        b23.at[i,'HT']='PEC Zwolle'
    elif b23.iloc[i]['AT']=='FC Zwolle':
        print('ok')
        b23.at[i,'AT']='PEC Zwolle'

ok


In [31]:
b23.loc[np.where(np.logical_or(b23['HT']=='FC Zwolle',b23['AT']=='FC Zwolle'))[0]]

,League,Date,Time,HT,AT,ODDH,ODDD,ODDA,ODD_25+,ODD_25-,ODD_BthScore_Yes,ODD_BthScore_No,ODD_1X,ODD_12,ODD_X2,Scrape_Date,Scrape_Time


In [13]:
np.unique(portal2['HT'])

array(['ADO Den Haag', 'AZ Alkmaar', 'Ajax', 'Emmen', 'FC Twente',
       'FC Utrecht', 'Feyenoord', 'Groningen', 'Heracles Almelo',
       'PEC Zwolle', 'PSV Eindhoven', 'RKC Waalwijk', 'Sparta Rotterdam',
       'VVV-Venlo', 'Vitesse Arnhem', 'Willem II'], dtype=object)

In [19]:
betano2 =Prepare__Df(betano1,season,l)

Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"


In [20]:
#sky2
sky_teams=(list(set(list(np.unique(list(sky2['HT'])))+list(np.unique(list(sky2['AT']))))))
betano_teams=list(set(list(np.unique(list(betano2['HT'])))+list(np.unique(list(betano2['AT'])))))

same=okok
same_sky = [same[i][0] for i in range(len(same))]
same_betano = [same[i][1] for i in range(len(same))]
missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
missing_betano = [betano_teams[i] for i in range(len(betano_teams)) if (betano_teams[i] not in same_betano)]

In [34]:
betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
Betano_TopOdds = Betano_odd(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)

What


In [33]:
Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)

In [35]:
Predix=Odds_Top2(Predix,Betano_TopOdds)
Predix

here we go
0
same [0]
1
same [1]
2
same [8]


,League,HT,AT,Date,Time,ODDH_TOP1,Time_1,ODDH_TOP2,Time_2,ODDH_TOP3,Time_3,ODDH_TOP4,Time_4,ODDH_TOP5,Time_5
0,Ligue_1,Paris Saint-Germain,Angers,2020-10-02,2000,1.19,143,1.19,128,1.19,128,1.19,128,1.15,176
1,Ligue_1,RC Lens,St Etienne,2020-10-03,1600,2.82,196,2.33,163,2.33,148,2.33,148,2.33,148
8,Ligue_1,Rennes,Reims,2020-10-04,1600,1.8,187,1.75,172,1.75,172,1.75,172,1.67,146


In [36]:
Predix

,League,HT,AT,Date,Time,ODDH_TOP1,Time_1,ODDH_TOP2,Time_2,ODDH_TOP3,Time_3,ODDH_TOP4,Time_4,ODDH_TOP5,Time_5
0,Ligue_1,Paris Saint-Germain,Angers,2020-10-02,2000,1.19,143,1.19,128,1.19,128,1.19,128,1.15,176
1,Ligue_1,RC Lens,St Etienne,2020-10-03,1600,2.82,196,2.33,163,2.33,148,2.33,148,2.33,148
8,Ligue_1,Rennes,Reims,2020-10-04,1600,1.8,187,1.75,172,1.75,172,1.75,172,1.67,146


In [30]:
def Odds_Top2(predix,bto):
    #junta à df dos jogos previstos(predix) a info sobre a evolução das odds(bto)
    predix = predix.drop(['ODDH_Aver.','ODDH_Aver.','ODDH_Aver.'],axis=1)
    predix = predix.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    bto = bto.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    i=0
    add_df = pd.DataFrame({})
    print('here we go')
    while i<len(predix):
        print(i)
        same = list(np.where(np.logical_and(
        np.logical_and(bto['HT']==predix.iloc[i]['HT'],bto['AT']==predix.iloc[i]['AT']),
        np.array(list(map(lambda x,y:days_diff(x,y),list(bto['Date']),[predix.iloc[i]['Date']]*len(list(bto['Date'])))))<=10
        ))[0])
        print('same',same)
        if len(same)==1:
            add_df = pd.concat([add_df,pd.DataFrame(bto.loc[same[0]]).T])
            i+=1
        else:
            print(len(same))
            raise Exception(':(, dont know what happened')
    return add_df

In [18]:
same

[['RC Lens', 'RC Lens'],
 ['Angers', 'Angers'],
 ['Montpellier', 'Montpellier'],
 ['Dijon', 'Dijon'],
 ['Rennes', 'Rennes'],
 ['Nimes', 'Nimes'],
 ['Monaco', 'Monaco'],
 ['Lille', 'Lille'],
 ['Lyon', 'Lyon'],
 ['Nice', 'Nice'],
 ['Nantes', 'Nantes'],
 ['Metz', 'FC Metz'],
 ['Lorient', 'FC Lorient'],
 ['Reims', 'Stade Reims'],
 ['St Etienne', 'Saint-Etienne'],
 ['Bordeaux', 'Bordeus'],
 ['Marseille', 'Marselha'],
 ['Paris Saint-Germain', 'PSG'],
 ['Strasbourg', 'Estrasburgo'],
 ['Brest', 'Stade Brestois 29']]

In [8]:
Betano_utn(sky2,betano2,l)

['Nantes', 'Bordeaux', 'Paris Saint-Germain', 'RC Lens', 'St Etienne', 'Monaco', 'Nice', 'Marseille', 'Angers', 'Lorient', 'Montpellier', 'Strasbourg', 'Rennes', 'Dijon', 'Reims', 'Metz', 'Brest', 'Lille', 'Nimes', 'Lyon']
Bordeaux? (write "passa" if not sure Or "reset" to restart) x


KeyboardInterrupt: 

In [6]:
op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//BetanoTeams_Ligue_1.pickle','rb')
okok = pickle.load(op)
op.close()

okok

[['RC Lens', 'RC Lens'],
 ['Angers', 'Angers'],
 ['Montpellier', 'Montpellier'],
 ['Dijon', 'Dijon'],
 ['Rennes', 'Rennes'],
 ['Nimes', 'Nimes'],
 ['Monaco', 'Monaco'],
 ['Lille', 'Lille'],
 ['Lyon', 'Lyon'],
 ['Nice', 'Nice'],
 ['Nantes', 'Nantes'],
 ['Metz', 'FC Metz'],
 ['Lorient', 'FC Lorient'],
 ['Reims', 'Stade Reims'],
 ['St Etienne', 'Saint-Etienne'],
 ['Bordeaux', 'Bordeus'],
 ['Marseille', 'Marselha'],
 ['Paris Saint-Germain', 'PSG'],
 ['Strasbourg', 'Estrasburgo'],
 ['Brest', 'Stade Brestois 29']]

In [7]:
Betano_utn(sky2,Betano_TopOdds,l)

['Nantes', 'Bordeaux', 'Paris Saint-Germain', 'RC Lens', 'St Etienne', 'Monaco', 'Nice', 'Marseille', 'Angers', 'Lorient', 'Montpellier', 'Strasbourg', 'Rennes', 'Dijon', 'Reims', 'Metz', 'Brest', 'Lille', 'Nimes', 'Lyon']
Bordeaux? (write "passa" if not sure Or "reset" to restart) Bordeaux
Paris Saint-Germain? (write "passa" if not sure Or "reset" to restart) x


KeyboardInterrupt: 

In [4]:
Betano_TopOdds.loc[Betano_TopOdds['League']=='Ligue_1']

,League,HT,AT,Date,Time,ODDH_TOP1,Time_1,ODDH_TOP2,Time_2,ODDH_TOP3,Time_3,ODDH_TOP4,Time_4,ODDH_TOP5,Time_5
9,Ligue_1,PSG,Angers,2020-10-02,2000,1.19,143,1.19,128,1.19,128,1.19,128,1.15,176
45,Ligue_1,RC Lens,Saint-Etienne,2020-10-03,1600,2.82,196,2.33,163,2.33,148,2.33,148,2.33,148
56,Ligue_1,Nice,Nantes,2020-10-03,2000,2.35,200,2.35,126,2.35,97,2.35,79,2.35,71
63,Ligue_1,Montpellier,Nimes,2020-10-04,1200,1.91,216,1.83,183,1.83,168,1.83,168,1.83,168
68,Ligue_1,Bordeus,Dijon,2020-10-04,1400,1.72,218,1.67,185,1.67,170,1.67,170,1.67,170
72,Ligue_1,Estrasburgo,Lille,2020-10-04,1400,3.35,170,3.35,170,3.35,170,3.30,144,3.2,185
73,Ligue_1,Stade Brestois 29,Monaco,2020-10-04,1400,3.30,170,3.30,170,3.30,170,3.20,185,3,144
74,Ligue_1,FC Metz,FC Lorient,2020-10-04,1400,2.35,144,2.30,170,2.30,170,2.30,170,2.2,185
82,Ligue_1,Rennes,Stade Reims,2020-10-04,1600,1.80,187,1.75,172,1.75,172,1.75,172,1.67,146
90,Ligue_1,Lyon,Marselha,2020-10-04,2000,1.87,191,1.83,176,1.83,176,1.83,176,1.83,150


In [6]:
All_Predix

,League,Date,Time,HT,AT,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
0,Ligue_1,2020-10-02,1900.0,Paris Saint-Germain,Angers,1.12,7.80,17.0
1,Ligue_1,2020-10-03,1500.0,RC Lens,St Etienne,2.10,3.25,3.4
2,Ligue_1,2020-10-04,1500.0,Rennes,Reims,1.67,3.40,5.3


In [ ]:
def Predix_League(PreHusky,l):
    if l == 'Ligue_1': #o modelo e os valores limite estão errados
        modelx = 'Husky_DFLL1_P2.pickle'
        TL = 60
        FL = 60
        clubs = H1.findall_clubs2_new(PreHusky,read=False)
        Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
        Husky = Husky0.tail(portal_len)
        rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
        F=pickle.load(rd0)
        lixo_Train=pickle.load(rd0)
        lixo_Test=pickle.load(rd0)
        T_df=pickle.load(rd0)
        rd0.close()
        print(Husky.columns)
        big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
        Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
        Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
        betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
        Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
        lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
        Predix=Odds_Top(Predix,Betano_TopOdds)
    elif l == 'Serie_A':
        modelx = 'Husky_DFLL1_P2.pickle'
        TL = 60
        FL = 60
        clubs = H1.findall_clubs2_new(PreHusky,read=False)
        Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
        Husky = Husky0.tail(portal_len)
        rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
        F=pickle.load(rd0)
        lixo_Train=pickle.load(rd0)
        lixo_Test=pickle.load(rd0)
        T_df=pickle.load(rd0)
        rd0.close()
        print(Husky.columns)
        big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
        Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
        Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
        betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
        Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
        lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
        Predix=Odds_Top(Predix,Betano_TopOdds
                        
                        
    elif l == 'Premier_League':
        modelx = 'Husky_DFLL1_P2.pickle'
        TL = 60
        FL = 60
        clubs = H1.findall_clubs2_new(PreHusky,read=False)
        Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
        Husky = Husky0.tail(portal_len)
        rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
        F=pickle.load(rd0)
        lixo_Train=pickle.load(rd0)
        lixo_Test=pickle.load(rd0)
        T_df=pickle.load(rd0)
        rd0.close()
        print(Husky.columns)
        big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
        Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
        Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
        betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
        Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
        lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
        Predix=Odds_Top(Predix,Betano_TopOdds)
                        
                        
    elif l == 'Bundesliga':
        modelx = 'Husky_DFLL1_P2.pickle'
        TL = 60
        FL = 60
        clubs = H1.findall_clubs2_new(PreHusky,read=False)
        Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
        Husky = Husky0.tail(portal_len)
        rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
        F=pickle.load(rd0)
        lixo_Train=pickle.load(rd0)
        lixo_Test=pickle.load(rd0)
        T_df=pickle.load(rd0)
        rd0.close()
        print(Husky.columns)
        big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
        Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
        Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
        betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
        Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
        lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
        Predix=Odds_Top(Predix,Betano_TopOdds)
                        
                        
    elif l == 'La_Liga':
        modelx = 'Husky_DFLL1_P2.pickle'
        TL = 60
        FL = 60
        clubs = H1.findall_clubs2_new(PreHusky,read=False)
        Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
        Husky = Husky0.tail(portal_len)
        rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
        F=pickle.load(rd0)
        lixo_Train=pickle.load(rd0)
        lixo_Test=pickle.load(rd0)
        T_df=pickle.load(rd0)
        rd0.close()
        print(Husky.columns)
        big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
        Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
        Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
        betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
        Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
        lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
        Predix=Odds_Top(Predix,Betano_TopOdds)
                        
                        
    elif l == 'Eredivisie':
        modelx = 'Husky_DFLL1_P2.pickle'
        TL = 60
        FL = 60
        clubs = H1.findall_clubs2_new(PreHusky,read=False)
        Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
        Husky = Husky0.tail(portal_len)
        rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
        F=pickle.load(rd0)
        lixo_Train=pickle.load(rd0)
        lixo_Test=pickle.load(rd0)
        T_df=pickle.load(rd0)
        rd0.close()
        print(Husky.columns)
        big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
        Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
        Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
        betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
        Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
        lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
        Predix=Odds_Top(Predix,Betano_TopOdds)
                        
                        
    elif l == 'Premiership':
        modelx = 'Husky_DFLL1_P2.pickle'
        TL = 60
        FL = 60
        clubs = H1.findall_clubs2_new(PreHusky,read=False)
        Husky0 = H1.join_husky(PreHusky,clubs,5,diff=False)
        Husky = Husky0.tail(portal_len)
        rd0=open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//ZModelos//'+modelx,'rb')
        F=pickle.load(rd0)
        lixo_Train=pickle.load(rd0)
        lixo_Test=pickle.load(rd0)
        T_df=pickle.load(rd0)
        rd0.close()
        print(Husky.columns)
        big__ = Big_dix(Husky,F,T_df,var=1,tree_list=[TL],forest_list=[FL],optimize=False)
        Predix = go_get(big__,mode=2,lista=['Tree_Forest','TL'+str(TL),'FL'+str(FL),'Predicted'])
        Predix = Predix.drop(['Index','Tree Score %','Tree_var%'+str(TL),'ForestPredix_'+str(FL)+'%','ForestReal_'+str(FL)+'%','Won/NotWon'],axis=1)
        betano_dfs2 = list(map(lambda x:pd.read_csv(Bdir+x),list(os.listdir(Bdir))))
        Betano_TopOdds = Betano_oddtime(betano_dfs2,odd='ODDH',x=5,sort=True,league=l)
        lixo2,Betano_TopOdds = Betano_utn(sky2,Betano_TopOdds,l)
        Predix=Odds_Top(Predix,Betano_TopOdds)
    else:
        print(l)
        raise Exception('Check League names')
    
    return Predix

In [14]:
import time
import os
import pandas as pd 
import numpy as np
import datetime as dt

In [15]:
def get_figures():
    #vai buscar as infos sobre o dinheiro
    done = False
    while not(done):
        print('Get, Create, Ignore Figures?')
        time.sleep(0.2)
        ok = input('------------>')
        if (ok=='Get') or (ok=='Create') or (ok=='Ignore'):
            done = True
        else:
            print('Invalid Key')
    if ok=='Get':
        #try:
        r = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//')
        DF = pd.read_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//Fig_DF.xlsx')
        return DF
    elif ok=='Create':
        done = False
        while not(done):
            print('First amount of money?')
            time.sleep(0.2)
            ok = input('----------> ')
            try: 
                ok2 = int(ok)
                done = True
            except:
                print('Invalid Key')
        DF = pd.DataFrame({
            'Money':[ok2],'Money9_15':[5000],'Money3':[300000],'Money1_Simple':[40000],'Money3_SO':[2500],
            'Money1_SOSimple':[1500],'Creation_Date':[dt.datetime.now()]
        })
        
        
        with pd.ExcelWriter("C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//Fig_DF.xlsx") as writer:
            DF.to_excel(writer, index=None)
            #writer.save()
            writer.close()
        with pd.ExcelWriter("C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//Fig_DF_BU.xlsx") as writer2:
            DF.to_excel(writer2, index=None)
            #writer2.save()
            writer2.close()
                
        return DF
    elif ok == 'Ignore':
        return False
    
    else:
        raise Exception('Weird Error')

In [39]:
def Update_Figures():
    Fig_DF = get_figures()
    try:
        if Fig_DF == False:
            raise Exception('Ignore is not allowed')
    except:
        pass
    lix = []
    done=True
    d=0
    while done:
        try:
            vars()['df'+str(d)] = pd.read_excel("C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//TheBets2.xlsx",d)
            lix += [vars()['df'+str(d)]]
            d+=1
        except:
            done=False #acontece quando já não há folhas no ficheiro Excel
    if lix[0].empty:
        print('There are no Bets Finished')
        print('Check to see if this is True')
        #df.to_excel("C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//TheBets2.xlsx",index=False)
        raise Exception('TheBets2 is empty')
    lix3 = [] #para o TheBets2
    lix2 = [] #para analisar
    for df in lix:
        dAtE,i = max([[df.iloc[i]['Date'],i] for i in range(len(df)) if dt.datetime==type(df.iloc[i]['Date'])])
        tImE = df.iloc[i]['Time']
        print(dAtE,tImE)
        if len(str(tImE))==4:
            h,m=int(str(tImE)[:2]),int(str(tImE)[2:])
        elif len(str(tImE))==3:
            h,m=int(str(tImE)[:1]),int(str(tImE)[1:])
        elif len(str(tImE))==2:
            h,m=0,int(tImE)
        else:
            print(tImE)
            raise Exception('WTF time data')
            
        dAtE = dAtE + dt.timedelta(hours=h+2,minutes=m) #+3 pq assim sabemos que o jogo já acabou
        print(dAtE)
        #print(df)
        if dAtE >= dt.datetime.now():
            lix3 += [df]
        else:
            lix2 += [df]
    Money = Fig_DF.iloc[-1]['Money']
    Money2 = Fig_DF.iloc[-1]['Money']
    fig_list=[]
    print(len(lix2),'lix2')
    for df in lix2:
        #print(len(lix2))
        print('A')
        print('A')
        print('A')
        print('A')
        print('A')
        fig_list2 = [0,0,0,0,0,0,0]
        d=0
        beg=0
        while d<len(df):
            print(d,beg,pd.isnull(df.iloc[d]['League']))
            if d>=1:
                if beg==0 and not(pd.isnull(df.iloc[d]['League'])) and pd.isnull(df.iloc[d-1]['League']):
                    beg = d
            if df.iloc[d]['Date']=='DATE':
                try:
                    multi=len(df[beg:(d+1)])
                except:
                    multi=len(df[beg:])
                print(beg,'beg',df.iloc[beg]['HT'])
                print(' ')
                print(' ')
                if multi>3:
                    Multi = True
                else:
                    Multi = False
                SO = 1==df.iloc[d]['SO'] #se há jogo com SuperODDS
                
                #podemos depois tentar substituir esta parte para ser automático
                i=beg
                perc = df.iloc[d-1]['Time'] #%de money para a aposta
                odd = df.iloc[d-1]['ODDH_Last1'] #odd da aposta
                if odd==0:
                    beg=0
                    d+=1
                else:
                    while i<=(d-2):
                        print(df.iloc[i]['HT'],' VS ',df.iloc[i]['AT'],'|| ODD: ',df.iloc[d-1]['ODDH_Last1'])
                        i+=1
                    done=True
                    while done:
                        try:
                            okok = int(input('Aposta Ganha? Sim(1) Não(0) '))
                            if okok==1 or okok==0:
                                done = False
                            else:
                                print('Invalid Key')
                        except:
                            print('Invalid Key')
                ###############################################################################
                   
                    bet_money = round((perc*Money)/100,2)
                    print(Money)
                    #Validar o bet_money
                    bm = True
                    while bm:
                        print(bet_money,' foi o €€ apostado? Sim(1) Não(0)')
                        bm2 = eval(input('-----> ')) 
                        if bm2 not in [0,1]:
                            print('Invalid Key')
                        else:
                            if bm2 == 1:
                                bm = False
                            else:
                                while bm:
                                    print('Então quanto €€ foi?')
                                    moz = eval(input('----> '))
                                    print('Tem a certeza?,',moz,type(moz))
                                    moz2 = eval(input('Sim(1) Não(0) '))
                                    if moz2 not in [0,1]:
                                        print('Invalid Key')
                                    else:
                                        if moz2 == 1:
                                            bet_money = moz
                                            bm = False
                    #--------------------------------------------------
                    Money2 = Money2 - bet_money
                    if okok==1: #Aposta ganha
                        gain = round(bet_money * odd,2)
                        Money2 = Money2 + gain
                    #fig_list2[0]=Money
                    fig_list2[1] += bet_money if (odd<=1.5) else 0
                    fig_list2[2] += bet_money
                    fig_list2[3] += bet_money if not(Multi) else 0
                    fig_list2[4] += bet_money if SO else 0
                    fig_list2[5] += bet_money if (SO and not(Multi)) else 0

                    beg=0
                    d+=1
            else:
                d+=1
        dAtE2,i2 = max([[df.iloc[i]['Date'],i] for i in range(len(df)) if dt.datetime==type(df.iloc[i]['Date'])])
        tImE = df.iloc[i2]['Time']
        if len(str(tImE))==4:
            h2,m2=int(str(tImE)[:2]),int(str(tImE)[2:])
        elif len(str(tImE))==3:
            h2,m2=int(str(tImE)[:1]),int(str(tImE)[1:])
        elif len(str(tImE))==2:
            h2,m2=0,int(tImE)
        else:
            print(tImE)
            raise Exception('WTF time data')
            
        dAtE2 = dAtE2 + dt.timedelta(hours=h2,minutes=m2) #+3 pq assim sabemos que o jogo já acabou        
        fig_list2[0] = Money2
        fig_list2[6] = dAtE2
        fig_list += [fig_list2]
        
    print(fig_list)
    raise
    for lis in fig_list:
        frame = pd.DataFrame({
            'Money':[lis[0]],'Money9_15':[lis[1]],'Money3':[lis[2]],'Money1_Simple':[lis[3]],'Money3_SO':[lis[4]],
            'Money1_SOSimple':[lis[5]],'Creation_Date':[lis[6]]
        })
        Fig_DF = pd.concat([Fig_DF,frame],sort=False)
    os.remove('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//Fig_DF_BU.xlsx')
    os.rename('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//Fig_DF.xlsx',
    'C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//Fig_DF_BU.xlsx')
    print(type(Fig_DF))
    time.sleep(1)
    Fig_DF=Fig_DF.sort_values(['Creation_Date'],ascending=[True]).reset_index(drop=True)
    print(Fig_DF['Creation_Date'])
    Fig_DF.to_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Figures_Storage//Fig_DF.xlsx',index=False)
    
    
    
    try:
        with pd.ExcelWriter("C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//TheBets2.xlsx") as writer:
            j=0
            for n, df in enumerate(lix3):
                if not(df.empty):
                    df.to_excel(writer, sheet_name='Apostas_'+str(j), index=None)
                    j+=1
            writer.save()
        writer.close()
    except:
        pass

In [42]:
Update_Figures()

Get, Create, Ignore Figures?
------------>Get
2020-11-06 00:00:00 2000
2020-11-06 22:00:00
1 lix2
A
A
A
A
A
0 0 False
1 0 False
2 0 True
3 0 True
0 beg RB Leipzig
 
 
RB Leipzig  VS  SC Freiburg || ODD:  5.17
Borussia Dortmund  VS  Bayern Munich || ODD:  5.17
Aposta Ganha? Sim(1) Não(0) 0
33.13
1.62  foi o €€ apostado? Sim(1) Não(0)
-----> 1
4 0 True
5 0 True
6 0 True
7 0 True
8 0 False
9 8 True
10 8 True
8 beg RKC Waalwijk
 
 
RKC Waalwijk  VS  Sparta Rotterdam || ODD:  2.3
Aposta Ganha? Sim(1) Não(0) 1
33.13
1.62  foi o €€ apostado? Sim(1) Não(0)
-----> 1
[[33.62, 0, 3.24, 1.62, 0, 0, datetime.datetime(2020, 11, 6, 20, 0)]]


RuntimeError: No active exception to reraise